In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os
import glob
import imageio.v2 as imageio

In [ ]:
import sys
from pathlib import Path

aqui_camino = Path.cwd()
cdg_fuente_camino = (aqui_camino / '..' / 'cdg_fuente').resolve()
sys.path.append(str(cdg_fuente_camino))

import bases
import galerkin_discontinuo 
import paso_de_tiempo
import graficos

In [ ]:
# Parametros de entrada

# Dominio espacial de la simulacion 
x_inicial = 0 # (m) cooordenda inicial del dominio
x_final = 10 # (m) coordenada final del dominio

# Parametros del metodo de elementos finitos
N_elementos = 6 # numero de elementos
N_nodos = 4 # numero de nodos por elemento (por simplicidad solo consideramos N_nodos >= 2)

# Dominio temporal 
n_pasos = 100 # numero de pasos temporales
t_total = 1 # (s) tiempo final

# Integracion numerica con cuadratura de Gauss
n_nodos_cuadratura_gauss = 20

In [ ]:
# Generando malla
longitud_elemento = (x_final - x_inicial) / N_elementos
malla = np.array([np.linspace(x_inicial + i * (x_final - x_inicial) / N_elementos, x_inicial + (i + 1) * (x_final - x_inicial) / N_elementos, N_nodos) for i in range(N_elementos)])      

# Condiciones iniciales
h = 1.0 + 0.1 * np.exp( - ( malla - 5.0 )**2 ) # Altura (m)
u = malla * 0.0 # Velocidad horizontal (m/s)

In [ ]:
cuadratura_de_gauss, pesos_de_gauss = np.polynomial.legendre.leggauss(n_nodos_cuadratura_gauss)
polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss = bases.generate_reference_space(malla, cuadratura_de_gauss)
matriz_de_masa_inversa = galerkin_discontinuo.calcula_inversa_matriz_de_masa(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, N_nodos)
matriz_de_rigidez = galerkin_discontinuo.calcula_matrix_de_rigidez(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss)

In [ ]:
# paso de tiempo
paso_t = np.array(t_total/n_pasos) 

# creando carpeta para guardar las imagenes
os.makedirs('imagenes', exist_ok=True)

# evolucionando en el tiempo la ecuacion diferencial
for numero_de_paso in np.arange(n_pasos+1):

    print(f'paso {numero_de_paso}')

    graficos.plot_simulation(malla, h, u, N_elementos, paso_t, numero_de_paso, display=False)

    dh_dt, du_dt = paso_de_tiempo.compute_dhdt_du_dt(h, u, matriz_de_rigidez, matriz_de_masa_inversa)

    h += dh_dt * paso_t
    u += du_dt * paso_t

print(f'Done')

In [ ]:
duracion_por_cuadro = 0.3  # segundos

files = sorted(glob.glob("./imagenes/plt*.png"))
files = sorted(files, key=lambda x: int(os.path.splitext(os.path.basename(x))[0][3:]))

frames = [imageio.imread(f) for f in files]
imageio.mimsave("imagenes/animacion.gif", frames, duration=duracion_por_cuadro)

display(Image(filename="imagenes/animacion.gif"))